### Задача: на основе данных выявить поведенчиские шаблоны пьющих и непьющих студентов, можно делать в Jupyter notebook

Набор данных: https://archive.ics.uci.edu/ml/datasets/student+performance

In [1]:
import pandas as pd
import numpy as np

#### Читаем файлы

```
# %load stud_data/student.txt

# Attributes for both student-mat.csv (Math course) and student-por.csv (Portuguese language course) datasets:
1 school - student's school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)
2 sex - student's sex (binary: "F" - female or "M" - male)
3 age - student's age (numeric: from 15 to 22)
4 address - student's home address type (binary: "U" - urban or "R" - rural)
5 famsize - family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)
6 Pstatus - parent's cohabitation status (binary: "T" - living together or "A" - apart)
7 Medu - mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
8 Fedu - father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
9 Mjob - mother's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
10 Fjob - father's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
11 reason - reason to choose this school (nominal: close to "home", school "reputation", "course" preference or "other")
12 guardian - student's guardian (nominal: "mother", "father" or "other")
13 traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14 studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15 failures - number of past class failures (numeric: n if 1<=n<3, else 4)
16 schoolsup - extra educational support (binary: yes or no)
17 famsup - family educational support (binary: yes or no)
18 paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19 activities - extra-curricular activities (binary: yes or no)
20 nursery - attended nursery school (binary: yes or no)
21 higher - wants to take higher education (binary: yes or no)
22 internet - Internet access at home (binary: yes or no)
23 romantic - with a romantic relationship (binary: yes or no)
24 famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25 freetime - free time after school (numeric: from 1 - very low to 5 - very high)
26 goout - going out with friends (numeric: from 1 - very low to 5 - very high)
27 Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28 Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29 health - current health status (numeric: from 1 - very bad to 5 - very good)
30 absences - number of school absences (numeric: from 0 to 93)

# these grades are related with the course subject, Math or Portuguese:
31 G1 - first period grade (numeric: from 0 to 20)
31 G2 - second period grade (numeric: from 0 to 20)
32 G3 - final grade (numeric: from 0 to 20, output target)

Additional note: there are several (382) students that belong to both datasets . 
These students can be identified by searching for identical attributes
that characterize each student, as shown in the annexed R file.
```

In [2]:
df1 = pd.read_csv('stud_data/student-mat.csv', sep=';')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
school        395 non-null object
sex           395 non-null object
age           395 non-null int64
address       395 non-null object
famsize       395 non-null object
Pstatus       395 non-null object
Medu          395 non-null int64
Fedu          395 non-null int64
Mjob          395 non-null object
Fjob          395 non-null object
reason        395 non-null object
guardian      395 non-null object
traveltime    395 non-null int64
studytime     395 non-null int64
failures      395 non-null int64
schoolsup     395 non-null object
famsup        395 non-null object
paid          395 non-null object
activities    395 non-null object
nursery       395 non-null object
higher        395 non-null object
internet      395 non-null object
romantic      395 non-null object
famrel        395 non-null int64
freetime      395 non-null int64
goout         395 non-null int64
Dalc          395 no

In [3]:
df2 = pd.read_csv('stud_data/student-por.csv', sep = ';')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
school        649 non-null object
sex           649 non-null object
age           649 non-null int64
address       649 non-null object
famsize       649 non-null object
Pstatus       649 non-null object
Medu          649 non-null int64
Fedu          649 non-null int64
Mjob          649 non-null object
Fjob          649 non-null object
reason        649 non-null object
guardian      649 non-null object
traveltime    649 non-null int64
studytime     649 non-null int64
failures      649 non-null int64
schoolsup     649 non-null object
famsup        649 non-null object
paid          649 non-null object
activities    649 non-null object
nursery       649 non-null object
higher        649 non-null object
internet      649 non-null object
romantic      649 non-null object
famrel        649 non-null int64
freetime      649 non-null int64
goout         649 non-null int64
Dalc          649 no

#### В описании к данным указано, что некоторые студенты (382 студента) посещали обе группы, а так же указан набор полей, формирующих уникальную характристику студента. Будем их использовать для исключения из общего списка студентов, которые ходят на оба курса, что избавит от дублирования основных характеристик.

In [4]:
mf = ["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]

#### В списке курса по математике определяем индексы студентов, посещающих курсы португальского языка. 

#### В процессе слияния была выявлена ошибка в описаниии набора данных. Набор колонок, указанных в файле student-merge.R, не формирует уникальную характристику студента. В примере ниже видно, что формирующие уникальность поля совпадают, а остальные разнятся.

In [5]:
df1_ind = df1.merge(df2, how='inner', on=mf, right_index=True)
df1_ind.tail(4)[1::-1].T

,389,389
school,MS,MS
sex,F,F
age,18,18
address,U,U
famsize,GT3,GT3
Pstatus,T,T
Medu,1,1
Fedu,1,1
Mjob,other,other
Fjob,other,other


In [6]:
# определяем индексы студентов обоих курсов
stud_all_cources = df1.merge(df2, how='inner', on=mf, right_index=True).index

# удаляем повторяющиеся индексы
stud_all_cources = stud_all_cources.drop_duplicates() 

df1_uniq_stud = df1.drop(stud_all_cources)
df1_uniq_stud.index

# df1_ind.loc[df1_ind.duplicated()]

Int64Index([128, 161, 162, 163, 165, 292, 304, 305, 307, 309, 310, 311, 312,
            313, 314, 315, 336, 340, 360, 361, 370, 376, 390, 392, 394],
           dtype='int64')

#### Объединяем уникальные записи студентов в один список.

In [7]:
df = pd.concat([df2, df1_uniq_stud], ignore_index=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 33 columns):
school        674 non-null object
sex           674 non-null object
age           674 non-null int64
address       674 non-null object
famsize       674 non-null object
Pstatus       674 non-null object
Medu          674 non-null int64
Fedu          674 non-null int64
Mjob          674 non-null object
Fjob          674 non-null object
reason        674 non-null object
guardian      674 non-null object
traveltime    674 non-null int64
studytime     674 non-null int64
failures      674 non-null int64
schoolsup     674 non-null object
famsup        674 non-null object
paid          674 non-null object
activities    674 non-null object
nursery       674 non-null object
higher        674 non-null object
internet      674 non-null object
romantic      674 non-null object
famrel        674 non-null int64
freetime      674 non-null int64
goout         674 non-null int64
Dalc          674 no

#### Считаем пьющими всех, у кого общее потребление алкоголя больше 2 

In [9]:
alco = df[df['Dalc'] + df['Walc'] > 2]
len(alco)

423

In [10]:
no_alco = df[df['Dalc'] + df['Walc'] <= 2]
len(no_alco)

251

In [11]:
# alco = alco.drop(['Dalc', 'Walc'], axis=1)
# no_alco = no_alco.drop(['Dalc', 'Walc'], axis=1)

#### Определяем наиболее часто встречаемые значения характеристик

In [12]:
alco_mff = []
for c in alco.columns:
    alco_mff.append(alco[c].value_counts().idxmax())
alco_mff = pd.DataFrame([alco_mff], columns=alco.columns, index=['alco'])

In [13]:
no_alco_mff = []
for c in no_alco.columns:
    no_alco_mff.append(no_alco[c].value_counts().idxmax())
no_alco_mff = pd.DataFrame([no_alco_mff], columns=no_alco.columns, index=['no_alco'])

#### Наиболее часто встречаемые значения характеристик пьющих и непьющих студентов 

In [14]:
patterns = pd.concat([alco_mff, no_alco_mff])
patterns.T

,alco,no_alco
school,GP,GP
sex,F,F
age,17,16
address,U,U
famsize,GT3,GT3
Pstatus,T,T
Medu,4,2
Fedu,2,2
Mjob,other,other
Fjob,other,other


### Определяем пьющих и трезвенников

#### Категоризируем строковые значения

In [15]:
df_cat = df.copy()

In [16]:
cat_columns = df_cat.select_dtypes(include=['object']).columns 
cat_columns

Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic'],
      dtype='object')

In [17]:
for c in cat_columns: # конвертируем все колонки-обьекты в категории
    df_cat[c] = df_cat[c].astype('category')

In [18]:
# сохраним категории
categories = {}
for c in cat_columns:
    categories[c] = [v for v in df_cat[c].cat.categories]

categories

{'Fjob': ['at_home', 'health', 'other', 'services', 'teacher'],
 'Mjob': ['at_home', 'health', 'other', 'services', 'teacher'],
 'Pstatus': ['A', 'T'],
 'activities': ['no', 'yes'],
 'address': ['R', 'U'],
 'famsize': ['GT3', 'LE3'],
 'famsup': ['no', 'yes'],
 'guardian': ['father', 'mother', 'other'],
 'higher': ['no', 'yes'],
 'internet': ['no', 'yes'],
 'nursery': ['no', 'yes'],
 'paid': ['no', 'yes'],
 'reason': ['course', 'home', 'other', 'reputation'],
 'romantic': ['no', 'yes'],
 'school': ['GP', 'MS'],
 'schoolsup': ['no', 'yes'],
 'sex': ['F', 'M']}

In [19]:
df_cat[cat_columns] = df_cat[cat_columns].apply(lambda x: x.cat.codes) # заменим на категории

In [20]:
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 33 columns):
school        674 non-null int8
sex           674 non-null int8
age           674 non-null int64
address       674 non-null int8
famsize       674 non-null int8
Pstatus       674 non-null int8
Medu          674 non-null int64
Fedu          674 non-null int64
Mjob          674 non-null int8
Fjob          674 non-null int8
reason        674 non-null int8
guardian      674 non-null int8
traveltime    674 non-null int64
studytime     674 non-null int64
failures      674 non-null int64
schoolsup     674 non-null int8
famsup        674 non-null int8
paid          674 non-null int8
activities    674 non-null int8
nursery       674 non-null int8
higher        674 non-null int8
internet      674 non-null int8
romantic      674 non-null int8
famrel        674 non-null int64
freetime      674 non-null int64
goout         674 non-null int64
Dalc          674 non-null int64
Walc          674 non

In [21]:
df_cat.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,18,1,0,0,4,4,0,4,...,4,3,4,1,1,3,4,0,11,11
1,0,0,17,1,0,1,1,1,0,2,...,5,3,3,1,1,3,2,9,11,11
2,0,0,15,1,1,1,1,1,0,2,...,4,3,2,2,3,3,6,12,13,12
3,0,0,15,1,0,1,4,2,1,3,...,3,2,2,1,1,5,0,14,14,14
4,0,0,16,1,0,1,3,3,2,2,...,4,3,2,1,2,5,0,11,13,13


#### Считаем пьющими всех, у кого общее потребление алкоголя больше 2 

In [22]:
alco_labels = df_cat.apply(lambda x: 1 if x['Dalc'] + x['Walc'] > 2 else 0, axis=1)

In [23]:
alco_cat = df_cat[df_cat['Dalc'] + df_cat['Walc'] > 2]
len(alco_cat)

423

In [24]:
no_alco_cat = df_cat[df_cat['Dalc'] + df_cat['Walc'] <= 2]
len(no_alco_cat)

251

In [25]:
df_cat_to_fit = df_cat.drop(['Dalc', 'Walc'], axis=1)

In [26]:
df_cat_to_fit.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout',
       'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

#### Разбиваем данные на сегменты для обучения и предсказания

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    df_cat_to_fit,
    alco_labels,
    test_size=0.10,
    random_state=42
)

### Обучаем модели 

#### Logistic regression

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
lr = LogisticRegression()

In [31]:
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

lr_pred

array([1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1])

#### Decision tree

In [32]:
from sklearn import tree

In [33]:
dt_clf = tree.DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [34]:
dtree_pred = dt_clf.predict(X_test)
dtree_pred

array([0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0])

#### Random forest classifier

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
rf_clf = RandomForestClassifier(n_estimators=500, max_depth=5, random_state=0)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [37]:
forest_pred = rf_clf.predict(X_test)
forest_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### Metrics

In [38]:
from sklearn import metrics

#### Logistic regression

In [39]:
metrics.f1_score(y_test, lr_pred)

0.75

In [40]:
metrics.accuracy_score(y_test, lr_pred)

0.67647058823529416

#### Decision tree

In [41]:
metrics.f1_score(y_test, dtree_pred)

0.72527472527472536

In [42]:
metrics.accuracy_score(y_test, dtree_pred)

0.63235294117647056

#### Random forest classifier

In [43]:
metrics.f1_score(y_test, forest_pred)

0.74285714285714299

In [44]:
metrics.accuracy_score(y_test, forest_pred)

0.6029411764705882